In [ ]:
%run ../commons/imports_legal.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
create_database('blp_dataset')
create_table('products_tmp_amazon')

# == Files config ==
FILE_FOLDER = r"D:\legal_data\amazon_2020.csv"
fdata=pd.read_csv(Path(FILE_FOLDER))
IMG_FOLDER_DEST = r"D:\images\_amazon"

In [ ]:
fdata.columns

In [ ]:
# Deleting unwanted columns
del_list = ['Uniq Id', 'Brand Name', 'Asin', 'Category', 'Upc Ean Code', 'List Price', 'Selling Price', 'Quantity', 
            'Model Number', 'Product Specification', 'Technical Details', 'Shipping Weight', 'Product Dimensions', 'Variants',
            'Sku', 'Product Url', 'Stock', 'Product Details', 'Dimensions', 'Color', 'Ingredients', 'Direction To Use', 
            'Is Amazon Seller', 'Size Quantity Variant', 'Product Description']

fdata = fdata.drop(del_list, axis = 1)

fdata.columns

In [ ]:
def clear_text(text):
    clear_content = re.sub(r'[\t\r\n]', ' ', text)
    clear_content = re.sub(' +',' ', clear_content).lstrip()

    return clear_content

In [ ]:
def insert_product(row):   
    encoded_string = row['About Product'].encode('utf-8', "ignore")
    description = encoded_string.decode()
    
    description = description.replace("Make sure this fits by entering your model number. | ", "")
    description = re.sub('[👍🌍📱✅【】👐🔬⚗️⚙️🧜😎🔴🔵🙌🌿🎁🦄💃👜⭐💦]', '', description)
    description = clear_text(description.replace(' | ', ''))
    
    query = "INSERT INTO products_tmp_amazon (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
    values = (row['Product Name'], description, "Amazon - 2020", "f")

    cursor.execute(query, values)
    conn.commit()

    return cursor.lastrowid

In [ ]:
for index, row in fdata.iterrows():
    if row['Product Name'] == "NaN" or type(row['Product Name']) is not str: row['Product Name'] = ""
    if row['About Product'] == "NaN" or type(row['About Product']) is not str: row['About Product'] = ""

    if row['Product Name'] is not "" and row['About Product'] is not "":
        product_id = insert_product(row)

        image = row['Image'].split(sep="|")[0]

        folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}.jpeg")

        try:
            file = urllib.request.urlretrieve(image, folder_dest)
        except:
            print(f"Product {product_id}: Error retrieving image ({image})")